In [ ]:
# аксесcор cat
# позволяет получать информацию о своих значениях и преобразовывать их. 
# Например, с помощью атрибута этого аксессора categories мы можем получить список уникальных категорий в столбце Regionname:

In [1]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
# display(melb_data.head())
melb_df = melb_data.copy()
# display(melb_df.head())
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
#melb_df.head()
total_rooms = melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
#display(total_rooms)
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea']/total_rooms
#display(melb_df['MeanRoomsSquare'])
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
#display(melb_df['AreaRatio'])
price_square = melb_df['Price']**2
#display(price_square)

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
# print(melb_df['Date'])

year_sold = melb_df['Date'].dt.year
#print('год продажи', year_sold)
#print('min yearsold', year_sold.min())
#print('max yearsold', year_sold.max())
#print('mode of sold', year_sold.mode()[0])

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
melb_df = melb_df.drop(['YearBuilt'], axis=1)
#display(melb_df.head())

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
#print(weekend_count, 'объектов недвижимости было продано в выходные')

# В наших данных есть столбец с адресами объектов недвижимости
# check количество уникальных значений в столбце с помощью метода nunique()
#print('количество уникальных значений в столбце', melb_df['Address'].nunique())

# давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар)
# Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address
#print(melb_df['Address'].loc[177])
#print(melb_df['Address'].loc[1812])
#print(melb_df['Address'].loc[9001])

# Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    exclude_list = ['N', 'S', 'W', 'E'] # Создаём список географических пометок exclude_list.
    address_list = address.split(' ') # Метод split() разбивает строку на слова по пробелу, получаем список слов в строке 
    street_type = address_list[-1] # Обрезаем список, оставляя в нём только последний элемент, потенциальный подтип улицы
    if street_type in exclude_list: # Делаем проверку на то, что полученный подтип является географической пометкой at exclude_list
        street_type = address_list[-2] # Если переменная street_type является географической пометкой change на второй элемент с конца 
    return street_type
# Теперь применим эту функцию к столбцу c адресом 
# передадим функцию get_street_type в аргумент метода столбца apply()
street_types = melb_df['Address'].apply(get_street_type)
#display(street_types)
# функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу

# мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось
#print(street_types.nunique())

popular_stypes = street_types.value_counts().nlargest(10).index
#print(popular_stypes)

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'others')
#display(melb_df['StreetType'])

melb_df = melb_df.drop('Address', axis=1)
#display(melb_df.head())

def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
#print(round(melb_df[(melb_df['Weekend'] == 1)]['Price'].mean()))

popular_seler = melb_df['SellerG'].value_counts().nlargest(49).index
# заменяем значения улиц, не попавших в список популярных на строку 'other'
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seler else 'other') 
a = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min() 
b = melb_df[melb_df['SellerG'] == 'other']['Price'].min() 
#print(round(a/b, 1))

# ВАЖНО!!!! ВЫУЧИТЬ!!!
# Давайте определим число уникальных категорий в каждом столбце нашей таблицы melb_df. 
# Для этого создадим вспомогательную таблицу unique_counts:
# создаём пустой список, в который будем добавлять кортежи: имя столбца, количество уникальных значений в нём и тип столбца
unique_list = [] 
# В цикле перебираем имена столбцов, которые получаем с помощью атрибута columns
for col in melb_df.columns: # В переменной col на каждой итерации находятся имена столбцов
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её, даём названия его столбцам: Column_Name, Num_unique и Type.
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True) # Сортируем таблицу по столбцу: by = 'Num_unique' в порядке возрастания количества уникальных элементов с помощью метода sort_values() 
# выводим её на экран
#display(unique_counts)

# Сделаем преобразование столбцов к типу данных category:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # В цикле перебираем имена столбцов, и, если число уникальных категорий меньше заданного порога и имён столбцов нет в списке cols_to_exclude, то с помощью метода astype() приводим столбец к типу данных category.
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())

In [2]:
# с помощью атрибута этого аксессора categories мы можем получить список уникальных категорий в столбце Regionname
print(melb_df['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


In [3]:
# атрибут codes
# посмотрим, каким образом столбец кодируется в виде чисел в памяти компьютера.
display(melb_df['Regionname'].cat.codes)

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

In [6]:
# rename_categories() 
# переименовать текущие значения категорий
# нужно передать словарь, ключи которого — старые имена категорий, а значения — новые
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']